In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import sklearn 

# Loading/Visualizing Data

In [12]:
# obtaining condition labels
labels_path = '../../data/medical_abstracts_TC_corpus/medical_tc_labels.csv'
df_labels = pd.read_csv(labels_path)
df_labels

,condition_label,condition_name
0,1,neoplasms
1,2,digestive system diseases
2,3,nervous system diseases
3,4,cardiovascular diseases
4,5,general pathological conditions


In [40]:
# might be useful for mapping and visualization
df_labels.to_dict(orient='list')

{'condition_label': [1, 2, 3, 4, 5],
 'condition_name': ['neoplasms',
  'digestive system diseases',
  'nervous system diseases',
  'cardiovascular diseases',
  'general pathological conditions']}

In [24]:
# loading training data
train_path = '../../data/medical_abstracts_TC_corpus/medical_tc_train.csv'
df_train = pd.read_csv(train_path)
df_train.head()

,condition_label,medical_abstract
0,5,Tissue changes around loose prostheses. A cani...
1,1,Neuropeptide Y and neuron-specific enolase lev...
2,2,"Sexually transmitted diseases of the colon, re..."
3,1,Lipolytic factors associated with murine and h...
4,3,Does carotid restenosis predict an increased r...


In [26]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11550 entries, 0 to 11549
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   condition_label   11550 non-null  int64 
 1   medical_abstract  11550 non-null  object
dtypes: int64(1), object(1)
memory usage: 180.6+ KB


In [25]:
# loading test data
test_path = '../../data/medical_abstracts_TC_corpus/medical_tc_test.csv'
df_test = pd.read_csv(test_path)
df_test.head()

,condition_label,medical_abstract
0,3,Obstructive sleep apnea following topical orop...
1,5,Neutrophil function and pyogenic infections in...
2,5,A phase II study of combined methotrexate and ...
3,1,Flow cytometric DNA analysis of parathyroid tu...
4,4,Paraneoplastic vasculitic neuropathy: a treata...


In [27]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2888 entries, 0 to 2887
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   condition_label   2888 non-null   int64 
 1   medical_abstract  2888 non-null   object
dtypes: int64(1), object(1)
memory usage: 45.3+ KB
